# Active Semi-Supervised Learning training strategy

In this demo we will show how to perform an active semi-supervised learning strategy for the training of a deep learning neural network which aims to perform the GBM segmentation on TEM images.
In this demo we will set generic folders for the images and related masks, which mimic as much as possible the dataset used in the published paper.
Thus, the proposed code is ready-to-use with your custom dataset and it could be easily adapted also for other segmentation purposes.

The model used for the image segmentation is a classical EfficientNet-b3 architecture.
The detailed description about the model architecture could be found in the work of Curti et al. [1](TODO).

We used Tensorflow library for the model implementation, so be sure to have installed all the required package before the use of this script.

First of all, we need to import the required libraries and define the common variables and path for the use of the segmentation model.
The below script could be used for **all** the ASSL rounds of training, with minimum chaning in the below global variables.
Take care about the code comments for the correct usage of the script in different rounds.
You must take care about the folder tree in which the data (images and masks) are stored.
In this demo we assume a folder tree described as:

```bash
data/
├── gbm_patches_round0
├── gbm_masks_round0
├── gbm_images
├── gbm_masks
├── validation_patches_pred_round0
└── validation_masks_pred_round0
```

where:
* `gbm_patches_round0` contains the entire set of available images in the dataset;
* `gbm_masks` contains the entire set of **validated** masks in the dataset;
* `gbm_images_round0` contains the images to use during the current (round 0) round of ASSL training;
* `gbm_masks_round0` contains the masks to use during the current (round 0) round of ASSL training;
* `validation_patches_pred_round0` will be filled by the images with the predictions overlayed for the ASSL validation;
* `validation_masks_pred_round0` will be filled by the predictions of the trained models.

In [ ]:
import os

# define the current round number
ASSL_ROUND = 0
# define the batch-size to use during the training
BATCH = 8
# define the directory in which the whole DB of images are stored
ALL_IMAGE_FOLDER = './data/gbm_patches'
# define the directory in which the whole DB of (validated!!) masks are stored
ALL_MASKS_FOLDER = './data/gbm_masks'
# define the directory in which the images are stored
TRAIN_IMAGE_FOLDER = f'./data/gbm_patches_round{ASSL_ROUND:d}'
# define the directory in which the masks are stored
TRAIN_MASKS_FOLDER = f'./data/gbm_masks_round{ASSL_ROUND:d}'
# define the directory in which the predictions will be saved for the validation
PRED_IMAGE_FOLDER = f'./data/validation_patches_pred_round{ASSL_ROUND:d}'
# define the directory in which the predictions will be saved
PRED_MASKS_FOLDER = f'./data/validation_masks_pred_round{ASSL_ROUND:d}'

# crete the prediction folder if needed
os.makedirs(PRED_IMAGE_FOLDER, exist_ok=True)
os.makedirs(PRED_MASKS_FOLDER, exist_ok=True)

# define the output weight file for the best model checkpint
OUT_WEIGHT_FILE = f'./checkpoints/model_round{ASSL_ROUND:d}.h5'

To monitor also the development of the ASSL strategy, we can take a look at the statistics related to the considered data against the totality.
In each ASSL round we will split the available round-data into a training set (90%) and a validation set (10%): the training set will be used for the tuning of the model parameters, while the remaining validation set will be used for the monitoring of the model performances in a set of independent images.
For sake of completeness, we will check all the relevant stats for the round evaluation with the following log:

In [ ]:
from glob import glob

all_files = glob(f'{ALL_IMAGE_FOLDER}/*')
print(f'All available images: {len(all_files):d}')
round_files = glob(f'{TRAIN_IMAGE_FOLDER}/*')
round_perc = len(round_files)/len(all_files)*100
print(f'Images used at round {ASSL_ROUND:d}: {len(round_files):d} ({round_perc:.3f}%)')
train_perc = round(len(round_files)*.9)
test_perc = round(len(round_files)*.1)
print(f'Images used as training at round {ASSL_ROUND:d}: {train_perc:d} (90%)')
print(f'Images used as test at round {ASSL_ROUND:d}: {test_perc:d} (10%)')

Since we want to perform also a data-augmentation step during the training, we will use the APIs of the Tensorflow library for the correct management of the images and desired transformations.

In [ ]:
import tensorflow as tf

# define the data-augmentation parameters
augmentation_params = {
    'rotation_range':360,       # all possible rotations
    'width_shift_range':0.0,    # avoid width shift
    'height_shift_range':0.0,   # avoid height shift
    'fill_mode':'reflect',      # use reflection to fill the augmented image
    'shear_range':0.,           # avoid shear
    'zoom_range':0.,            # avoid zoom
    'horizontal_flip':True,     # perform horizontal flip of the image
    'vertical_flip':True,       # perform vertical flip of the image
    'cval':0.,                  # just the constant value for the augmented background
    'validation_split':0.1,     # set the validation set as the 10% of the entire set of data
}

# define the data augmentation models for images and masks
# NOTE: both images and masks must be rescaled into [0, 1] range for 
# the correct use of the segmentation model!
image_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
    **augmentation_params,
    rescale = 1./255
)
masks_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
    **augmentation_params,
    rescale = 1./255
)

Now we need to define the data-loader strategy for the images/masks.
Using the global information about the folder tree and the data-augmentation models, we can use the Tensorflow APIs as follow.

In [ ]:
# we fixed the dimensionality of the input as 256x256
IMG_SIZE = 256

# define the training parameters for the data loader
train_params = {
    'target_size':(IMG_SIZE, IMG_SIZE),  # resize shape of the input
    'class_mode':'input',  # this is the input of the model
    'batch_size':BATCH,    # set the batch size
    'shuffle':True,        # enable shuffling of the data
    'seed':42,             # fix the random seed for the reproducibility
}

# define the data loader for the training data (aka images and masks)
train_image_generator = image_augmentation.flow_from_directory(
    directory=TRAIN_IMAGE_FOLDER,  # set the folder of the images
    **train_params,                # set the training parameters
    color_mode='grayscale',        # the images are in GRAY fmt
    classes=[''],                  # there are no classes
    subset='training'              # this subset is the training one (aka the 90% of the data)
)
train_masks_generator = masks_augmentation.flow_from_directory(
    directory=TRAIN_MASKS_FOLDER,  # set the folder of the masks
    **train_params,                # set the training parameters
    color_mode='grayscale',        # the masks are in binary format
    classes=[''],                  # there are no classes
    subset='training'              # this subset is the training one (aka the 90% of the data)
)

# define the data loader for the validation data (aka images and masks)
val_image_generator = image_augmentation.flow_from_directory(
    directory=TRAIN_IMAGE_FOLDER,  # the validation images belongs to the same folder of the training ones
    **train_params,                # set the training parameters
    color_mode='grayscale',        # the images are in GRAY fmt
    classes=[''],                  # there are no classes
    subset='validation'            # this subset is the validation one (aka the 10% of the data)
)
val_masks_generator = masks_augmentation.flow_from_directory(
    directory=TRAIN_MASKS_FOLDER,  # the validation images belongs to the same folder of the training ones
    **train_params,                # set the training parameters
    color_mode='grayscale',        # the masks are in binary format
    classes=[''],                  # there are no classes
    subset='validation'            # this subset is the validation one (aka the 10% of the data)
)

# NOTE: all the data (training and validation) belongs to the same directory
# and the internal subdivision is guaranteed by the subset keyword of the
# tensorflow function.

# Since we want to combine images and masks into a series of pairs, we
# can use a pre-processing on the data loader generator to obtain the
# correct input for our model

from itertools import itemgetter

# create training pairs
train_generator = zip(map(itemgetter(0), (train_image_generator)), 
                      map(itemgetter(0), (train_masks_generator))
                     )
# create validation pairs
validation_generator = zip(map(itemgetter(0), (val_image_generator)), 
                           map(itemgetter(0), (val_masks_generator))
                          )

Now we can build the model setting the missing training parameters, i.e. the loss function and the optimization strategy.
Since the model architecture is already defined in the deepskin package, we can directly import it and setting the training parameters.

**NOTE:** Since we are inside an ASSL training round, the model weights **must** be initialized as random at each round!

The definition of the loss function and metrics used in the original model were combinations of native functions.
For their implementation we used the code provided by the `segmentation_models` package (ref. [here](https://github.com/qubvel/segmentation_models)).
Importing this library, we defined the loss function as combination of Dice score and Binary Focal Loss functions.
The monitoring of the model performances is defined using the standard IoU score and the F-score.

In [ ]:
import segmentation_models as sm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D

# define the model architecture
model = sm.models.efficientnetb3(trainable=True)
inp = Input(shape=(IMG_SIZE, IMG_SIZE, 1))
l1 = Conv2D(3, (1, 1))(inp)

out = model(l1)

model = Model(inp, out, name=model.name)

# define the model optimizer
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-4,
    beta_1=0.9, beta_2=0.999,
    epsilon=1e-7,
    amsgrad=False,
    name='Adam'
)

# define the loss function
loss = sm.losses.DiceLoss() + (1 * sm.losses.BinaryFocalLoss())

# define the metric functions for the model evaluation along
# the training epochs
iou_score = sm.metrics.IOUScore(threshold=0.5)
fscore = sm.metrics.FScore(threshold=0.5)

# set the training parameters
model.compile(
    optimizer=optimizer, 
    loss=loss,
    metrics=[iou_score, fscore],
    run_eagerly=False,
)

When everything about the model parameters is decided and fixed, we can start the training step, enabling all the utilities provided by the Tensorflow.

In [ ]:
# fit model
history = model.fit(
    x=train_generator, y=None, # define the input data generators
    batch_size=BATCH,          # set the batch size
    epochs=100,                # set the maximum number of epochs to perform 
    steps_per_epoch=train_image_generator.n // BATCH, # define the number of steps for each 
                                                      # epoch according to the data generator
    callbacks=[
                                                      # define the callback for the model checkpoint
                                                      # setting the output file in which save the best results
                                                      # given by the minimum of loss obtained
        tf.keras.callbacks.ModelCheckpoint(
            OUT_WEIGHT_FILE, 
            save_weights_only=True, 
            save_best_only=True, 
            mode='min'
        ),
                                                      # define the callback for the reduction of learning rate
                                                      # when a plateau of performances is achieved
        tf.keras.callbacks.ReduceLROnPlateau(),
                                                      # define the callback for the early stopping of the training
                                                      # if there are no improvements in the validation loss for 50
                                                      # epochs
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            min_delta=1e-4, patience=10, 
            verbose=True,
            mode='auto', baseline=None, 
            restore_best_weights=True
        ),
    ],
    validation_data=validation_generator,             # set the data validation generator
    validation_steps=val_image_generator.n // BATCH,  # define the number of steps for each
                                                      # validation according to the data generator
    initial_epoch=0,                                  # set the initial epoch counter
    validation_freq=1,                                # enable the validation at each epoch
    max_queue_size=10,                                # queue of data to use
    workers=1,                                        # number of threads to use
    use_multiprocessing=False,                        # disable multi-processing
    shuffle=True,                                     # enable the shuffling of the data at each epoch
    verbose=1                                         # set verbosity level of the training
)

At the end of the training the model will achieved the best performances of the current ASSL round.
An important step for the monitoring of the performances is the visualization of the obtained results, expressed in terms of metric parameters and loss along the training epochs.

In [ ]:
import pylab as plt
import seaborn as sns
import matplotlib.patches as mpatches

# define the legends for the plots
fig1_lbl = [ mpatches.Patch(facecolor='blue', label='Train Loss', edgecolor='k', linewidth=2),
             mpatches.Patch(facecolor='orange', label='Val Loss', edgecolor='k', linewidth=2)
           ]

fig2_lbl = [ mpatches.Patch(facecolor='blue', label='IoU train score', edgecolor='k', linewidth=2),
             mpatches.Patch(facecolor='orange', label='IoU val score', edgecolor='k', linewidth=2)
           ]

fig3_lbl = [ mpatches.Patch(facecolor='blue', label='F1-score train', edgecolor='k', linewidth=2),
             mpatches.Patch(facecolor='orange', label='F1-score val', edgecolor='k', linewidth=2)
           ]

epochs = np.arange(len(history.history['loss']))

with sns.plotting_context('paper', font_scale=2):
    fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(30, 8))
    loss = sns.lineplot(x=epochs, y=history.history['loss'],
                        markers=True, dashes=False, 
                        ax=ax1)
    val_loss = sns.lineplot(x=epochs, y=history.history['val_loss'],
                            markers=True, dashes=False, 
                            ax=ax1)
    ax1.set_ylabel('Mask Loss values')
    sns.despine(ax=ax1, offset=10, top=True, right=True, bottom=False, left=False)
    ax1.legend(handles=fig1_lbl, loc='upper right')
    
    
    loss = sns.lineplot(x=epochs, y=history.history['iou_score'],
                        markers=True, dashes=False, 
                        ax=ax2)
    val_loss = sns.lineplot(x=epochs, y=history.history['val_iou_score'],
                            markers=True, dashes=False, 
                            ax=ax2)
    ax2.set_ylabel('IoU loss')
    sns.despine(ax=ax2, offset=10, top=True, right=True, bottom=False, left=False)
    ax2.legend(handles=fig2_lbl, loc='best')
    
    loss = sns.lineplot(x=epochs, y=history.history['f1-score'],
                        markers=True, dashes=False, 
                        ax=ax3)
    val_loss = sns.lineplot(x=epochs, y=history.history['val_f1-score'],
                            markers=True, dashes=False, 
                            ax=ax3)
    ax3.set_ylabel('F1-score loss')
    sns.despine(ax=ax3, offset=10, top=True, right=True, bottom=False, left=False)
    ax3.legend(handles=fig3_lbl, loc='best')

Now we need to take care about the prediction of the model on the new data, i.e. the data which belongs to the whole dataset.
Since we set the restoring of the best model parameters at the end of the training epochs, we can directly apply the model on the new images, sampled on the dataset folder.

**NOTE:** since the image needs some pre-processing step, we need to manually apply the required sequence of instruction before inserting it in the model.

The correct management of the ASSL training strategy requires the validation of the entire set of images in the dataset at each round of training.
Therefore, if you want to check the effectiveness of the ASSL training, the list of files for the model prediction **must** be collected from the `ALL_IMAGE_FOLDER` folder.